# Python version of test_SPDEDRIFT.cpp

In [ ]:
import gstlearn as gl
import matplotlib.pyplot as plt
import gstlearn.plot as gp
import os 

verbose = False
flagSPDE = True
ndim = 2

In [ ]:
filename = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "temperatures.ascii")
temperatures = gl.Db.createFromNF(filename,verbose)
temperatures.setLocator("January_temp", gl.ELoc.Z)
temperatures.display()
gp.point(temperatures,"January_temp")

In [ ]:
filename = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "grid.ascii")
grid = gl.DbGrid.createFromNF(filename,verbose)
grid.display()
gp.grid(grid,"Elevation")

Calculate the variogram

In [ ]:
v = gl.VarioParam()
dir1 = gl.DirParam(2,18,25)
v.addDir(dir1)
vario = gl.Vario(v,temperatures)
md = gl.Model.createFromDb(temperatures)
md.addDrift(gl.Drift1(md.getContext()))
md.addDrift(gl.DriftF(md.getContext()))
vario.compute(model=md)
vario.display()
ax=gp.vario(vario)

Load the (same) variogram from a neutral file

In [ ]:
filename = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "vario.ascii")
vario = gl.Vario.createFromNF(filename,verbose)
vario.display()
gp.vario(vario)

Fit the variogram of residuals in a model having drifts

In [ ]:
model = gl.Model.createFromDb(temperatures)
model.setDriftList(md.getDriftList())

structs = [gl.ECov.NUGGET,gl.ECov.BESSEL_K,gl.ECov.BESSEL_K]
structs = [gl.ECov.NUGGET,gl.ECov.BESSEL_K]

consNug = gl.ConsItem.define(gl.EConsElem.SILL,0, type = gl.EConsType.UPPER,value = 0.1)

cons1P = gl.ConsItem.define(gl.EConsElem.PARAM,1, type = gl.EConsType.EQUAL,value = 1)
cons1Rm = gl.ConsItem.define(gl.EConsElem.RANGE,1, type = gl.EConsType.LOWER,value = 100)
cons1RM = gl.ConsItem.define(gl.EConsElem.RANGE,1, type = gl.EConsType.UPPER,value = 350)

cons2P = gl.ConsItem.define(gl.EConsElem.PARAM,2, type = gl.EConsType.EQUAL,value = 2)
cons2Rm = gl.ConsItem.define(gl.EConsElem.RANGE,2, type = gl.EConsType.LOWER,value = 100)
cons2RM = gl.ConsItem.define(gl.EConsElem.RANGE,2, type = gl.EConsType.UPPER,value = 400)

a = gl.Constraints()
a.addItem(consNug)
a.addItem(cons1P)
a.addItem(cons1Rm)
a.addItem(cons1RM)
a.addItem(cons2P)
a.addItem(cons2Rm)
a.addItem(cons2RM)

err = model.fit(vario,structs,constraints=a)
model.display()
ax = gp.varmod(vario,model,title="Vario of the residuals")

Reload the (same) model from a neutral file

In [ ]:
filename = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "model.ascii")
model = gl.Model.createFromNF(filename,verbose)
model.display()
ax = gp.varmod(vario,model,title="Vario of the residuals")

In [ ]:
spde = gl.SPDE(model,grid,temperatures,gl.ESPDECalcMode.KRIGING)
coeffs = spde.getCoeffs()
ax = gp.correlation(temperatures, "Elevation", "*temp", asPoint=True)
if len(coeffs)>1:
    plt.plot([0,400], [coeffs[0],coeffs[0]+coeffs[1]*400])

In [ ]:
spde.compute()
spde.query(grid)
ax = gp.grid(grid,usesel=True,figsize=[7,7])

In [ ]:
neighU = gl.NeighUnique.create(ndim, False);
gl.kriging(temperatures, grid, model, neighU);
ax = gp.grid(grid,usesel=True,figsize=[7,7])

In [ ]:
ax = gp.correlation(grid,"*estim","spde*",asPoint=True, diagLine=True)